In [9]:
from goSim import GoEnv
import numpy as np
import copy

In [10]:
class Wrapper():
    def __init__(self, board_size, komi):
        self.player_color = 'black'
        self.board_size = board_size
        self.komi = komi
        self.actions = []
        self.env = GoEnv(player_color=self.player_color, observation_type='image3c', illegal_move_mode="raise", board_size=board_size, komi=komi)
        # init board
        self.obs_t = self.env.reset()
        self.states = [self.obs_t]
        
    def step(self, action):
        self.actions.append(action)
        self.obs_t, r_t, done, info, cur_score = self.env.step(action)
        self.states.append(self.obs_t)
        if(self.env.player_color == 1):
            self.env.set_player_color(2)
        else:
            self.env.set_player_color(1)
        return self.obs_t, r_t, done, info, cur_score
    
    def actions_for_current_player(self,obs):
        actions = []
        for i in range((self.board_size)**2):
            if(self.env.is_legal_action(obs,i,self.env.player_color)):
                actions.append(i)
        actions.append((self.board_size)**2) #pass action
        actions.append((self.board_size)**2 + 1) #resign action
        return actions
    
def copy_curr(game):
    cpyEnv = Wrapper(game.board_size,game.komi)
    for i in range(len(game.actions)):
        a = cpyEnv.step(game.actions[i])
    return cpyEnv
    
        

In [11]:
newB = Wrapper(13,7.5)
for i in range(169):
    if(newB.env.player_color==1):
        obs_t, r_t, done, info, cur_score = newB.step(i+1)
    else:
        obs_t, r_t, done, info, cur_score = newB.step(i-1)
print(cur_score)
# obs_t, r_t, done, info, cur_score = newB.step(0)
# print(cur_score)
# obs_t, r_t, done, info, cur_score = newB.step(1)
# print(cur_score)
# obs_t, r_t, done, info, cur_score = newB.step(5)
# print(cur_score)
# obs_t, r_t, done, info, cur_score = newB.step(13)
# print(cur_score)

1
0
3
2
5
4
7
6
9
8
11
10
13
12
15
14
17
16
19
18
21
20
23
22
25
24
27
26
29
28
31
30
33
32
35
34
37
36
39
38
41
40
43
42
45
44
47
46
49
48
51
50
53
52
55
54
57
56
59
58
61
60
63
62
65
64
67
66
69
68
71
70
73
72
75
74
77
76
79
78
81
80
83
82
85
84
87
86
89
88
91
90
93
92
95
94
97
96
99
98
101
100
103
102
105
104
107
106
109
108
111
110
113
112
115
114
117
116
119
118
121
120
123
122
125
124
127
126
129
128
131
130
133
132
135
134
137
136
139
138
141
140
143
142
145
144
147
146
149
148
151
150
153
152
155
154
157
156
illegal_move, considered as Pass
159
158
illegal_move, considered as Pass
161
160
illegal_move, considered as Pass
163
162
illegal_move, considered as Pass
165
164
illegal_move, considered as Pass
167
166
illegal_move, considered as Pass
-161.5


In [12]:
cpy = copy_curr(newB)

1
0
3
2
5
4
7
6
9
8
11
10
13
12
15
14
17
16
19
18
21
20
23
22
25
24
27
26
29
28
31
30
33
32
35
34
37
36
39
38
41
40
43
42
45
44
47
46
49
48
51
50
53
52
55
54
57
56
59
58
61
60
63
62
65
64
67
66
69
68
71
70
73
72
75
74
77
76
79
78
81
80
83
82
85
84
87
86
89
88
91
90
93
92
95
94
97
96
99
98
101
100
103
102
105
104
107
106
109
108
111
110
113
112
115
114
117
116
119
118
121
120
123
122
125
124
127
126
129
128
131
130
133
132
135
134
137
136
139
138
141
140
143
142
145
144
147
146
149
148
151
150
153
152
155
154
157
156
illegal_move, considered as Pass
159
158
illegal_move, considered as Pass
161
160
illegal_move, considered as Pass
163
162
illegal_move, considered as Pass
165
164
illegal_move, considered as Pass
167
166
illegal_move, considered as Pass


In [21]:
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)


class Stone(object):
    def __init__(self, board, point, color):
        self.board = board
        self.point = point
        self.color = color
        self.group = self.find_group()

    def remove(self):
        self.group.stones.remove(self)
        del self

    @property
    def neighbors(self):
        neighboring = [(self.point[0] - 1, self.point[1]),
                       (self.point[0] + 1, self.point[1]),
                       (self.point[0], self.point[1] - 1),
                       (self.point[0], self.point[1] + 1)]
        for point in neighboring:
            if not 0 < point[0] < 14 or not 0 < point[1] < 14:
                neighboring.remove(point)
        return neighboring

    @property
    def liberties(self):
        liberties = self.neighbors
        stones = self.board.search(points=self.neighbors)
        for stone in stones:
            liberties.remove(stone.point)
        return liberties

    def find_group(self):
        groups = []
        stones = self.board.search(points=self.neighbors)
        for stone in stones:
            if stone.color == self.color and stone.group not in groups:
                groups.append(stone.group)
        if not groups:
            group = Group(self.board, self)
            return group
        else:
            if len(groups) > 1:
                for group in groups[1:]:
                    groups[0].merge(group)
            groups[0].stones.append(self)
            return groups[0]

    def __str__(self):
        return str(self.point[0]-1) + str(self.point[1]-1)


class Group(object):
    def __init__(self, board, stone):
        self.board = board
        self.board.groups.append(self)
        self.stones = [stone]
        self.liberties = None

    def merge(self, group):
        for stone in group.stones:
            stone.group = self
            self.stones.append(stone)
        self.board.groups.remove(group)
        del group

    def remove(self):
        while self.stones:
            self.stones[0].remove()
        self.board.groups.remove(self)
        del self

    def update_liberties(self):
        liberties = []
        for stone in self.stones:
            for liberty in stone.liberties:
                liberties.append(liberty)
        self.liberties = set(liberties)
        if len(self.liberties) == 0:
            self.remove()

    def __str__(self):
        return str([str(stone) for stone in self.stones])


class Board(object):
    def __init__(self):
        self.groups = []
        self.next = BLACK

    def search(self, point=None, points=[]):
        stones = []
        for group in self.groups:
            for stone in group.stones:
                if stone.point == point and not points:
                    return stone
                if stone.point in points:
                    stones.append(stone)
        return stones

    def turn(self):
        if self.next == BLACK:
            self.next = WHITE
            return BLACK
        else:
            self.next = BLACK
            return WHITE
    def update_liberties(self, added_stone=None):
        for group in self.groups:
            if added_stone:
                if group == added_stone.group:
                    continue
            group.update_liberties()
        if added_stone:
            added_stone.group.update_liberties()


In [24]:
board = Board()
for i in range(2):
    x = i+1
    y = i+1
    stone = board.search(point=(x, y))
    added_stone = Stone(board, (x, y), board.turn())
    board.update_liberties(added_stone)
    print(len(board.groups))
    for j in range(len(board.groups)):
        print(board.groups[j],board.groups[j].stones[0].color)

1
['00'] (0, 0, 0)
2
['00'] (0, 0, 0)
['11'] (255, 255, 255)
